In [1]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [6]:
class DNN(Model):
    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        self.l1 = Dense(hidden_dim, activation='sigmoid')
        self.l2 = Dense(hidden_dim, activation = 'sigmoid')
        self.l3 = Dense(hidden_dim, activation = 'sigmoid')
        self.l4 = Dense(output_dim, activation = 'softmax')
        self.ls = [self.l1, self.l2, self.l3, self.l4]
    
    def call(self, x):
        for layer in self.ls:
            x = layer(x)
        
        return x

In [7]:
mnist = datasets.mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = (x_train.reshape(-1, 784)/255).astype(np.float32)
x_test  = (x_test.reshape(-1, 784)/ 255).astype(np.float32)
t_train = np.eye(10)[t_train].astype(np.float32)
t_test  = np.eye(10)[t_test].astype(np.float32)

In [8]:
model = DNN(200, 10)

In [10]:
criterion = losses.CategoricalCrossentropy()
optimizer = optimizers.SGD(learning_rate = 0.01)
train_loss = metrics.Mean()
train_acc = metrics.CategoricalAccuracy()

def compute_loss(t, y):
    return criterion(t, y)

def train_step(x, t):
    with tf.GradientTape() as tape:
        preds = model(x)
        loss = compute_loss(t, preds)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss)
    train_acc(t, preds)
    return loss

epochs = 30
batch_size = 100
n_batches = x_train.shape[0] // batch_size
for epoch in range(epochs):
    x_, t_ = shuffle(x_train, t_train)
    
    for batch in range(n_batches):
        start = batch * batch_size
        end   = start + batch_size
        train_step(x_[start:end], t_[start:end])
    
    print('epoch:{}, loss:{:.3}, acc:{:.3f}'.format(epoch+1, train_loss.result(), train_acc.result()))

epoch:1, loss:2.3, acc:0.115
epoch:2, loss:2.3, acc:0.121
epoch:3, loss:2.3, acc:0.126
epoch:4, loss:2.29, acc:0.132
epoch:5, loss:2.29, acc:0.139
epoch:6, loss:2.29, acc:0.147
epoch:7, loss:2.28, acc:0.158
epoch:8, loss:2.28, acc:0.173
epoch:9, loss:2.27, acc:0.190
epoch:10, loss:2.27, acc:0.209
epoch:11, loss:2.26, acc:0.229
epoch:12, loss:2.24, acc:0.247
epoch:13, loss:2.22, acc:0.264
epoch:14, loss:2.18, acc:0.282
epoch:15, loss:2.14, acc:0.300
epoch:16, loss:2.09, acc:0.320
epoch:17, loss:2.04, acc:0.339
epoch:18, loss:1.98, acc:0.357
epoch:19, loss:1.93, acc:0.375
epoch:20, loss:1.88, acc:0.391
epoch:21, loss:1.83, acc:0.407
epoch:22, loss:1.79, acc:0.423
epoch:23, loss:1.74, acc:0.437
epoch:24, loss:1.7, acc:0.451
epoch:25, loss:1.66, acc:0.464
epoch:26, loss:1.62, acc:0.477
epoch:27, loss:1.59, acc:0.489
epoch:28, loss:1.56, acc:0.500
epoch:29, loss:1.52, acc:0.511
epoch:30, loss:1.49, acc:0.522


In [12]:
test_loss = metrics.Mean()
test_acc = metrics.CategoricalAccuracy()

def test_step(x, t):
    preds = model(x)
    loss = compute_loss(t, preds)
    test_loss(loss)
    test_acc(t, preds)
    return loss

test_step(x_test, t_test)

print('test_loss:{:.3f}, test_acc:{:.3f}'.format(
test_loss.result(),
test_acc.result()
))

test_loss:0.587, test_acc:0.830
